In [ ]:
dbutils.fs.mount(
  source = "wasbs://gold@ilab9788543873.blob.core.windows.net",
  mount_point = "/mnt/ilab9788543873/gold",
  extra_configs = {"fs.azure.account.key.ilab9788543873.blob.core.windows.net":"TAyrrfaiCQ86vkgSXyStufowrMVdk4T45mVw6TNcFFJocR6pTXy6ZMSUUTeeh5FpNTsBVAbVqdwk+AStHA3x1g=="})

In [13]:
import pandas as pd

# Path to the Parquet file
file_path = "dbfs:/mnt/ilab9788543873/gold/final_joined_df2.parquet"

# Read the Parquet file into a pandas DataFrame
df = pd.read_parquet(file_path, engine='pyarrow')

# Display the first few rows of the DataFrame
print(df.head())


               lens_id date_published kind  \
0  041-096-994-434-303     2019-12-31   B2   
1  041-096-994-434-303     2019-12-31   B2   
2  041-096-994-434-303     2019-12-31   B2   
3  041-096-994-434-303     2019-12-31   B2   
4  041-096-994-434-303     2019-12-31   B2   

                                                text  \
0  A system to allocate resources via information...   
1  A system to allocate resources via information...   
2  A system to allocate resources via information...   
3  A system to allocate resources via information...   
4  A system to allocate resources via information...   

                                          claim_text    applicant_name  \
0  1. A system to manage resource allocation via ...  ALEGEUS TECH LLC   
1  1. A system to manage resource allocation via ...  ALEGEUS TECH LLC   
2  1. A system to manage resource allocation via ...  ALEGEUS TECH LLC   
3  1. A system to manage resource allocation via ...  ALEGEUS TECH LLC   
4  1. A system t

In [15]:
import pandas as pd

# Assuming df is your DataFrame

# Custom aggregation function
def custom_agg(series):
    unique_vals = series.dropna().unique()
    return ';'.join(map(str, unique_vals))

# Group by lens_id and aggregate with the custom function
agg_df = df.groupby('lens_id').agg(custom_agg).reset_index()


In [16]:
agg_df.head(2)

,lens_id,date_published,kind,text,claim_text,applicant_name,invention_title,inventor_name,cited_lens_id,cited_jurisdiction,...,cited_phase,nplcit_lens_id,nplcit_text,patcit_jurisdiction,patcit_kind,main_cpc,sub_cpc,main_cpc_description,sub_cpc_description,jurisdiction
0,000-012-609-013-177,2019-08-29,A1,Systems and methods are provided that leverage...,1 . A method of determining data about natural...,EMERSON PROCESS MAN POWER & WATER SOLUTIONS INC,"Method, apparatus, server, and systems of time...",BALLAGUET JEAN-PIERRE R,178-421-345-280-222,CN,...,APP,,"“Components of GE Predictor.TM. Services,” GE ...",,,H,H04,ELECTRICITY,ELECTRIC COMMUNICATION TECHNIQUE,US
1,000-023-766-650-923,2021-05-13,A1,Disclosed is a process and system for training...,1 . An information technology system comprisin...,BECTON DICKINSON CO,Security door locking structure based on Inter...,LEE HO YOUN,000-382-027-825-034,KR,...,APP,,,US,A,Y,Y04,GENERAL TAGGING OF NEW TECHNOLOGICAL DEVELOPME...,INFORMATION OR COMMUNICATION TECHNOLOGIES HAVI...,US


In [18]:
from tabulate import tabulate

print(tabulate(agg_df.head(), headers='keys', tablefmt='grid'))


+----+---------------------+------------------+--------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [30]:
agg1 = agg_df.head(5)

In [36]:
from transformers import pipeline

# Initialize sentiment-analysis pipeline
nlp = pipeline("sentiment-analysis")

# Truncate the text to fit within token limit
agg_df['truncated_text'] = agg_df['text'].apply(lambda x: x[:500])
agg_df['sentiment'] = agg_df['truncated_text'].apply(lambda x: nlp(x)[0]['label'])


def split_and_aggregate_sentiment(text, nlp):
    # Split text into chunks of size 500
    chunks = [text[i:i+500] for i in range(0, len(text), 500)]
    
    # Get sentiments for each chunk
    sentiments = [nlp(chunk)[0]['label'] for chunk in chunks]
    
    # Aggregate the sentiments (for simplicity, we'll just return the most common sentiment)
    from collections import Counter
    return Counter(sentiments).most_common(1)[0][0]

agg_df['sentiment'] = agg_df['text'].apply(lambda x: split_and_aggregate_sentiment(x, nlp))



No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [37]:
def map_sentiment_to_category(sentiment):
    if sentiment == "POSITIVE":
        return ["Trust", "Transparent", "Empowerment", "Integration", "Diversity", "Individualism", "Acceleration"]
    elif sentiment == "NEGATIVE":
        return ["Mistrust", "Opaque", "Disempowerment", "Fragmentation", "Homogeneity", "Collectivism", "Deceleration"]
    else:
        return ["Neutral"] * 7

agg_df['categories'] = agg_df['sentiment'].apply(map_sentiment_to_category)


In [38]:
agg_df.head()

,lens_id,date_published,kind,text,claim_text,applicant_name,invention_title,inventor_name,cited_lens_id,cited_jurisdiction,...,patcit_jurisdiction,patcit_kind,main_cpc,sub_cpc,main_cpc_description,sub_cpc_description,jurisdiction,truncated_text,sentiment,categories
0,000-012-609-013-177,2019-08-29,A1,Systems and methods are provided that leverage...,1 . A method of determining data about natural...,EMERSON PROCESS MAN POWER & WATER SOLUTIONS INC,"Method, apparatus, server, and systems of time...",BALLAGUET JEAN-PIERRE R,178-421-345-280-222,CN,...,,,H,H04,ELECTRICITY,ELECTRIC COMMUNICATION TECHNIQUE,US,Systems and methods are provided that leverage...,POSITIVE,"[Trust, Transparent, Empowerment, Integration,..."
1,000-023-766-650-923,2021-05-13,A1,Disclosed is a process and system for training...,1 . An information technology system comprisin...,BECTON DICKINSON CO,Security door locking structure based on Inter...,LEE HO YOUN,000-382-027-825-034,KR,...,US,A,Y,Y04,GENERAL TAGGING OF NEW TECHNOLOGICAL DEVELOPME...,INFORMATION OR COMMUNICATION TECHNOLOGIES HAVI...,US,Disclosed is a process and system for training...,NEGATIVE,"[Mistrust, Opaque, Disempowerment, Fragmentati..."
2,000-052-978-540-155,2021-12-09,A1,Information indicating what technology and ser...,1 . A technology name and service name generat...,GUANGDONG XIANGYI SCIENCE AND TECH INFORMATION...,TECHNOLOGIES FOR PROCESSING NETWORK PACKETS IN...,MENAHEM GABBY,105-688-517-302-040;094-133-942-377-768;056-37...,US,...,,,H,H04,ELECTRICITY,ELECTRIC COMMUNICATION TECHNIQUE,US,Information indicating what technology and ser...,NEGATIVE,"[Mistrust, Opaque, Disempowerment, Fragmentati..."
3,000-067-543-894-474,2020-08-25,B1,Systems and methods detecting onboard sensor t...,1. A computer-implemented method in an electro...,ZIXCORP SYSTEMS INC,KONFIGURATIONSVERFAHREN UND -VORRICHTUNG FÜR L...,ZMUDZINSKI KRYSTOF,033-063-952-790-261;198-160-552-725-741;012-41...,US,...,,,B,B22,PERFORMING OPERATIONS; TRANSPORTING,CASTING; POWDER METALLURGY,US,Systems and methods detecting onboard sensor t...,NEGATIVE,"[Mistrust, Opaque, Disempowerment, Fragmentati..."
4,000-114-524-168-716,2019-01-22,B2,A method for generating a control signal to co...,1. A method for generating a control signal to...,FERRING BV,Video conversion technology,MAES STEPHANE H,,,...,WO,A1,A,A23,HUMAN NECESSITIES,"FOODS OR FOODSTUFFS; TREATMENT THEREOF, NOT CO...",US,A method for generating a control signal to co...,NEGATIVE,"[Mistrust, Opaque, Disempowerment, Fragmentati..."


In [45]:
from transformers import pipeline

# Initialize sentiment analysis pipeline
sentiment_pipeline = pipeline('sentiment-analysis')

def classify_text(text):
    result = sentiment_pipeline(text)[0]
    label = result['label']
    score = result['score']
    
    # Mapping sentiment to categories
    if label == 'POSITIVE':
        return {
            'Time Horizon': 'Medium term',
            'Opportunity-Challenge': 'Some opportunity',
            'Likelihood': 'Almost Certain',
            'Degree of Change': 'Medium Change',
            'Scale of Impact': 'Society'
        }
    elif label == 'NEGATIVE':
        return {
            'Time Horizon': 'Medium term',
            'Opportunity-Challenge': 'Some challenge',
            'Likelihood': 'Possible',
            'Degree of Change': 'Substantial Change',
            'Scale of Impact': 'Individual / Household'
        }
    else:  # NEUTRAL
        return {
            'Time Horizon': 'Medium term',
            'Opportunity-Challenge': 'Tension',
            'Likelihood': 'Probable',
            'Degree of Change': 'Partial Change',
            'Scale of Impact': 'Organisation'
        }

# Apply the function to the 'text' column
results = agg_df['truncated_text'].apply(classify_text)

# Map results to separate columns
agg_df['Time Horizon'] = results.apply(lambda x: x['Time Horizon'])
agg_df['Opportunity-Challenge'] = results.apply(lambda x: x['Opportunity-Challenge'])
agg_df['Likelihood'] = results.apply(lambda x: x['Likelihood'])
agg_df['Degree of Change'] = results.apply(lambda x: x['Degree of Change'])
agg_df['Scale of Impact'] = results.apply(lambda x: x['Scale of Impact'])


No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [46]:
agg_df.head()

,lens_id,date_published,kind,text,claim_text,applicant_name,invention_title,inventor_name,cited_lens_id,cited_jurisdiction,...,sub_cpc_description,jurisdiction,truncated_text,sentiment,categories,Time Horizon,Opportunity-Challenge,Likelihood,Degree of Change,Scale of Impact
0,000-012-609-013-177,2019-08-29,A1,Systems and methods are provided that leverage...,1 . A method of determining data about natural...,EMERSON PROCESS MAN POWER & WATER SOLUTIONS INC,"Method, apparatus, server, and systems of time...",BALLAGUET JEAN-PIERRE R,178-421-345-280-222,CN,...,ELECTRIC COMMUNICATION TECHNIQUE,US,Systems and methods are provided that leverage...,POSITIVE,"[Trust, Transparent, Empowerment, Integration,...",Medium term,Some opportunity,Almost Certain,Medium Change,Society
1,000-023-766-650-923,2021-05-13,A1,Disclosed is a process and system for training...,1 . An information technology system comprisin...,BECTON DICKINSON CO,Security door locking structure based on Inter...,LEE HO YOUN,000-382-027-825-034,KR,...,INFORMATION OR COMMUNICATION TECHNOLOGIES HAVI...,US,Disclosed is a process and system for training...,NEGATIVE,"[Mistrust, Opaque, Disempowerment, Fragmentati...",Medium term,Some challenge,Possible,Substantial Change,Individual / Household
2,000-052-978-540-155,2021-12-09,A1,Information indicating what technology and ser...,1 . A technology name and service name generat...,GUANGDONG XIANGYI SCIENCE AND TECH INFORMATION...,TECHNOLOGIES FOR PROCESSING NETWORK PACKETS IN...,MENAHEM GABBY,105-688-517-302-040;094-133-942-377-768;056-37...,US,...,ELECTRIC COMMUNICATION TECHNIQUE,US,Information indicating what technology and ser...,NEGATIVE,"[Mistrust, Opaque, Disempowerment, Fragmentati...",Medium term,Some challenge,Possible,Substantial Change,Individual / Household
3,000-067-543-894-474,2020-08-25,B1,Systems and methods detecting onboard sensor t...,1. A computer-implemented method in an electro...,ZIXCORP SYSTEMS INC,KONFIGURATIONSVERFAHREN UND -VORRICHTUNG FÜR L...,ZMUDZINSKI KRYSTOF,033-063-952-790-261;198-160-552-725-741;012-41...,US,...,CASTING; POWDER METALLURGY,US,Systems and methods detecting onboard sensor t...,NEGATIVE,"[Mistrust, Opaque, Disempowerment, Fragmentati...",Medium term,Some challenge,Possible,Substantial Change,Individual / Household
4,000-114-524-168-716,2019-01-22,B2,A method for generating a control signal to co...,1. A method for generating a control signal to...,FERRING BV,Video conversion technology,MAES STEPHANE H,,,...,"FOODS OR FOODSTUFFS; TREATMENT THEREOF, NOT CO...",US,A method for generating a control signal to co...,NEGATIVE,"[Mistrust, Opaque, Disempowerment, Fragmentati...",Medium term,Some challenge,Possible,Substantial Change,Individual / Household


In [47]:
columns = ['Time Horizon', 'Opportunity-Challenge', 'Likelihood', 'Degree of Change', 'Scale of Impact']

for col in columns:
    print(f"Value distribution for '{col}':")
    print(agg_df[col].value_counts())
    print("\n" + "-"*50 + "\n")


Value distribution for 'Time Horizon':
Time Horizon
Medium term    9546
Name: count, dtype: int64

--------------------------------------------------

Value distribution for 'Opportunity-Challenge':
Opportunity-Challenge
Some challenge      7961
Some opportunity    1585
Name: count, dtype: int64

--------------------------------------------------

Value distribution for 'Likelihood':
Likelihood
Possible          7961
Almost Certain    1585
Name: count, dtype: int64

--------------------------------------------------

Value distribution for 'Degree of Change':
Degree of Change
Substantial Change    7961
Medium Change         1585
Name: count, dtype: int64

--------------------------------------------------

Value distribution for 'Scale of Impact':
Scale of Impact
Individual / Household    7961
Society                   1585
Name: count, dtype: int64

--------------------------------------------------



In [48]:
pip install bert-extractive-summarizer

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9546 entries, 0 to 9545
Data columns (total 33 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   lens_id                9546 non-null   object
 1   date_published         9546 non-null   object
 2   kind                   9546 non-null   object
 3   text                   9546 non-null   object
 4   claim_text             9546 non-null   object
 5   applicant_name         9546 non-null   object
 6   invention_title        9546 non-null   object
 7   inventor_name          9546 non-null   object
 8   cited_lens_id          9546 non-null   object
 9   cited_jurisdiction     9546 non-null   object
 10  cited_kind             9546 non-null   object
 11  symbol                 9546 non-null   object
 12  grant_date             9546 non-null   object
 13  granted                9546 non-null   object
 14  patent_status          9546 non-null   object
 15  cited_phase          

In [58]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load the T5 tokenizer and model for summarization
tokenizer = AutoTokenizer.from_pretrained("t5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

# Create an empty list to store the summaries
summaries = []

# Use the head() method to take only the first 30 rows and then iterate over them
for index, row in agg_df.head(30).iterrows():
    text_to_summarize = row['text']

    # Tokenize the input text
    input_ids = tokenizer.encode("summarize: " + text_to_summarize, return_tensors="pt", max_length=1024, truncation=True)

    # Generate the summary
    summary_ids = model.generate(input_ids, max_length=150, min_length=30, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    # Append the summary to the list
    summaries.append(summary)

# The following will ensure that 'summary' column will have the generated summaries for the first 30 rows and NaN for the rest.
agg_df['summary'] = pd.Series(summaries)


In [61]:
df = agg_df

In [23]:
import numpy as np

# Possible values for each column
values_dict = {
    'Time Horizon': ['Near term', 'Medium term', 'Long term'],
    'Opportunity-Challenge': ['Some opportunity', 'Some challenge', 'Tension'],
    'Likelihood': ['Almost Certain', 'Possible', 'Probable'],
    'Degree of Change': ['Medium Change', 'Substantial Change', 'Partial Change'],
    'Scale of Impact': ['Society', 'Individual / Household', 'Organisation']
}

# Assign random values for each column
for column, values in values_dict.items():
    df[column] = np.random.choice(values, size=len(df))


In [26]:
# Possible values for the 'sentiment' column
sentiments = ['NEGATIVE', 'POSITIVE']

# Assign random values for the 'sentiment' column
df['sentiment'] = np.random.choice(sentiments, size=len(df))


In [28]:
# Create new columns based on the categories
df['Trust/Mistrust'] = np.where(df['categories'].str.contains('Trust'), 'Trust', 'Mistrust')
df['Transparency'] = np.where(df['categories'].str.contains('Transparent'), 'Transparent', 'Opaque')
df['Empowerment'] = np.where(df['categories'].str.contains('Empowerment'), 'Empowerment', 'Disempowerment')
df['Integration'] = np.where(df['categories'].str.contains('Integration'), 'Integration', 'Fragmentation')
df['Diversity'] = np.where(df['categories'].str.contains('Diversity'), 'Diversity', 'Homogeneity')
df['Individualism/Collectivism'] = np.where(df['categories'].str.contains('Individualism'), 'Individualism', 'Collectivism')
df['Pace'] = np.where(df['categories'].str.contains('Acceleration'), 'Acceleration', 'Deceleration')


In [32]:
import random

# Define the potential values for each new column
values_trust = ['Trust', 'Mistrust']
values_transparency = ['Transparent', 'Opaque']
values_empowerment = ['Empowerment', 'Disempowerment']
values_integration = ['Integration', 'Fragmentation']
values_diversity = ['Diversity', 'Homogeneity']
values_ind_coll = ['Individualism', 'Collectivism']
values_pace = ['Acceleration', 'Deceleration']

# Randomly assign values to the new columns
df['Trust/Mistrust'] = df['categories'].apply(lambda x: random.choice(values_trust))
df['Transparency'] = df['categories'].apply(lambda x: random.choice(values_transparency))
df['Empowerment'] = df['categories'].apply(lambda x: random.choice(values_empowerment))
df['Integration'] = df['categories'].apply(lambda x: random.choice(values_integration))
df['Diversity'] = df['categories'].apply(lambda x: random.choice(values_diversity))
df['Individualism/Collectivism'] = df['categories'].apply(lambda x: random.choice(values_ind_coll))
df['Pace'] = df['categories'].apply(lambda x: random.choice(values_pace))


In [ ]:
storage_account_name = "ilab9788543873"
container = "gold"
storage_account_key = "TAyrrfaiCQ86vkgSXyStufowrMVdk4T45mVw6TNcFFJocR6pTXy6ZMSUUTeeh5FpNTsBVAbVqdwk+AStHA3x1g=="

spark.conf.set(f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net", storage_account_key)

df.write.parquet(f"wasbs://{container}@{storage_account_name}.blob.core.windows.net/collapsed_df.parquet")